In [ ]:
library(gplots)
library(ggplot2)
library(dplyr)
library(WGCNA)
options(stringsAsFactors = FALSE)

In [ ]:
############################
#STEP1######################
############################

dt = read.csv("./source_data/WGCNA/batch_cor_wgcna_input.txt", sep = '\t')

datExpr0 = as.data.frame(t(dt[, -c(1)]));
names(datExpr0) = dt$ensembl_id;
rownames(datExpr0) = names(dt)[-c(1)];

gsg = goodSamplesGenes(datExpr0, verbose = 3);
gsg$allOK

if (!gsg$allOK)
{
  # Optionally, print the gene and sample names that were removed:
  if (sum(!gsg$goodGenes)>0)
    printFlush(paste("Removing genes:", paste(names(datExpr0)[!gsg$goodGenes], collapse = ", ")));
  if (sum(!gsg$goodSamples)>0)
    printFlush(paste("Removing samples:", paste(rownames(datExpr0)[!gsg$goodSamples], collapse = ", ")));
  # Remove the offending genes and samples from the data:
  datExpr0 = datExpr0[!gsg$goodSamples, gsg$goodGenes]
}


sampleTree = hclust(dist(datExpr0), method = "average");

sizeGrWindow(12,9)
par(cex = 0.9);
par(mar = c(0,4,2,0))

In [ ]:
par(mar = c(0,4,2,0))
plot(sampleTree, sub="", xlab="", cex.lab = 1.5, cex.axis = 1.5, cex.main = 1)
abline(h =72, col = "red")

In [ ]:
clust = cutreeStatic(sampleTree, cutHeight = 75, minSize = 24)
#clust = cutreeStatic(sampleTree, cutHeight = 65, minSize = 2)

table(clust)

keepSamples = (clust==1)
datExpr = datExpr0[keepSamples, ]
rownames(datExpr)

nGenes = ncol(datExpr)
nSamples = nrow(datExpr)

#####traitdata#####

traitData = read.csv("./source_data/WGCNA/clinical_wgcna.txt", sep = '\t')

traitData[ , "year_sampling" ] <- c(1,1,2,2,3,3,1,1,2,2,3,3,1,1,2,2,3,3,1,1,2,2,3,3)

#allTraits = traitData[, -c(3:7)]
allTraits = traitData
#allTraits = allTraits[, c(2, 11:36) ]

#allTraits = traitData
#dim(allTraits)
#names(allTraits)


Samples = rownames(datExpr)
#Samples
traitRows = match(Samples, allTraits$sample)
#traitRows
datTraits = allTraits[traitRows, -1];
#datTraits
rownames(datTraits) = allTraits[traitRows,1];
#datTraits = datTraits[,c(1)]
datTraits

collectGarbage();

sampleTree2 = hclust(dist(datExpr), method = "average")
traitColors = numbers2colors(datTraits, signed = FALSE);

sizeGrWindow(9, 5)

##png("/home/chm/analyzing_file_byR/RNA/SQuIRE_count_ELTC/figures/sample_dendrogram.png",width=2000,height=2000,res=300)
#plotDendroAndColors(sampleTree2, traitColors,
#                    groupLabels = names(datTraits),
#                    main = "Sample dendrogram and trait heatmap")
#
#save(datExpr, datTraits, file = "filtered_age_trait_dataInput.RData")
##dev.off()

In [ ]:
plotDendroAndColors(sampleTree2, traitColors,
                    groupLabels = names(datTraits),
                    main = "Sample dendrogram and trait heatmap")

save(datExpr, datTraits, file = "filtered_age_trait_dataInput.RData")

In [ ]:
############################
#STEP2######################
############################

#Skip if you use Rstudio
#enableWGCNAThreads() 

lnames = load(file = "filtered_age_trait_dataInput.RData");

lnames
powers = c(c(1:10), seq(from = 12, to=20, by=2))
powers

#sft = pickSoftThreshold(datExpr, powerVector = powers, verbose = 5)
sft = pickSoftThreshold(datExpr, powerVector = powers, verbose = 5, networkType = "unsigned")

In [ ]:
par(mfrow = c(1,2))
cex1 = 0.9

plot(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
     xlab="Soft Threshold (power)",ylab="Scale Free Topology Model Fit,signed R^2",type="n",
     main = paste("Scale independence"))
text(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
     labels=powers,cex=cex1,col="red")

abline(h=0.80,col="red")


plot(sft$fitIndices[,1], sft$fitIndices[,5],
     xlab="Soft Threshold (power)",ylab="Mean Connectivity", type="n",
     main = paste("Mean connectivity"))
text(sft$fitIndices[,1], sft$fitIndices[,5], labels=powers, cex=cex1,col="red")

In [ ]:
cor <- WGCNA::cor

net = blockwiseModules(datExpr, power = 7,
                       TOMType = "unsigned", minModuleSize = 30,
                       reassignThreshold = 0, mergeCutHeight = 0.3,
                       numericLabels = TRUE, pamRespectsDendro = FALSE,
                       saveTOMs = TRUE,
                       saveTOMFileBase = "mfascicularisTOM",
                       verbose = 3)


table(net$colors)

In [ ]:
#sizeGrWindow(30, 15)
mergedColors = labels2colors(net$colors)

plotDendroAndColors(net$dendrograms[[1]], mergedColors[net$blockGenes[[1]]],
                    "Module colors",
                    dendroLabels = FALSE, hang = 0.03,
                    addGuide = TRUE, guideHang = 0.05)

In [ ]:
moduleLabels = net$colors
moduleColors = labels2colors(net$colors)
MEs = net$MEs;
geneTree = net$dendrograms[[1]]
save(MEs, moduleLabels, moduleColors, geneTree,
     file = "filtered_age_networkConstruction-auto.RData")

In [ ]:
############################
#STEP3######################
############################

lnames = load(file = "filtered_age_trait_dataInput.RData")
lnames

lnames = load(file = "filtered_age_networkConstruction-auto.RData")
lnames

nGenes = ncol(datExpr)
nSamples = nrow(datExpr)


MEs0 = moduleEigengenes(datExpr, moduleColors)$eigengenes
MEs = orderMEs(MEs0)
moduleTraitCor = cor(MEs, datTraits, use = "p")
moduleTraitPvalue = corPvalueStudent(moduleTraitCor, nSamples)

In [ ]:
moduleTraitCor
write.table(moduleTraitCor, file='./source_data/WGCNA_ME.txt', sep='\t',col.names = T, row.names = T)

In [ ]:
#sizeGrWindow(100,20)

textMatrix = paste(signif(moduleTraitCor, 2), " (",signif(moduleTraitPvalue, 1), ")", sep = "")
dim(textMatrix) = dim(moduleTraitCor)

par(mar = c(2, 10, 4, 4))
labeledHeatmap(Matrix = moduleTraitCor,
               xLabels = names(datTraits),
               yLabels = names(MEs),
               ySymbols = names(MEs),
               colorLabels = FALSE,
               colors = blueWhiteRed(20),
               textMatrix = textMatrix,
               setStdMargins = FALSE,
               cex.text = 0.5,
               zlim = c(-1,1),
               main = paste("Module-trait relationships"))

In [ ]:
#################################################################
#AGE#
#####

for(i in list('greenyellow','turquoise')){
age = as.data.frame(datTraits$age)
names(age) = "weight"
modNames = substring(names(MEs), 3)

geneModuleMembership = as.data.frame(cor(datExpr, MEs, use = "p"))
MMPvalue = as.data.frame(corPvalueStudent(as.matrix(geneModuleMembership), nSamples))

names(geneModuleMembership) = paste("MM", modNames, sep="")
names(MMPvalue) = paste("p.MM", modNames, sep="")
geneTraitSignificance = as.data.frame(cor(datExpr, age, use = "p"))
GSPvalue = as.data.frame(corPvalueStudent(as.matrix(geneTraitSignificance), nSamples))
names(geneTraitSignificance) = paste("GS.", names(age), sep="")
names(GSPvalue) = paste("p.GS.", names(age), sep="")

module = i
column = match(module, modNames)
moduleGenes = moduleColors==module

verboseScatterplot(abs(geneModuleMembership[moduleGenes, column]),
                   abs(geneTraitSignificance[moduleGenes, 1]),
                   xlab = paste("Module Membership in", module, "module"),
                   ylab = "Gene significance for age",
                   main = paste("Module membership vs. gene significance\n"),
                   cex.main = 1.2, cex.lab = 1.2, cex.axis = 1.2, col = "black", bg=i ,pch = 21, abline=TRUE)
    
length(names(datExpr)[moduleColors==i])
genes <- names(datExpr)[moduleColors==i]

write.table(genes, file = paste0("./source_data/WGCNA/DEmRNA_age_",i,"_WGCNA.txt"), col.names = F, row.names = F)
}